In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-09-07 09:49:54


In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
MaxOpenProcesses = 15
Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool'
Project_GDB  = Project_DIR + '\\2.Project_GDB.gdb'
ContractsDir = Project_DIR + '\\4.Contracts_AnalysisFiles'
DocumentsDir = Project_DIR + '\\5.Documents'
PythonDir    = Project_DIR + '\\6.Python_Codes'
ReceivedDir  = Project_DIR + '\\7.ReceivedData'

LayersDir    = DocumentsDir + '\\Layers'
XML_Dir      = ReceivedDir + '\\HSIP Candidate Applications'

APRX            = LayersDir + '\\HSIP_Project_Template.aprx'
SegSymbology    = LayersDir + '\\SegSymbology.lyrx'
IntSymbology    = LayersDir + '\\IntSymbology.lyrx'
IntSymbology2   = LayersDir + '\\IntStateSymbology.lyrx'
BeforeSymbology = LayersDir + '\\Crash_Before_Sev.lyrx'
AfterSymbology  = LayersDir + '\\Crash_After_Sev.lyrx'

# Geocoded projects
HSIP_Int  = Project_GDB + '\\WPPS_Int'
HSIP_Seg  = Project_GDB + '\\WPPS_Seg'

HSIP_Path    = ReceivedDir + '\\HSIP_AllApprovedProjects.xlsm'
WPPS_Ob_Path = ReceivedDir + '\\HSIP Contract and Obligation Data.xlsx'
WPPS_Pr_Path = ReceivedDir + '\\HSIP Project Programmed Data.xlsx'
SPF_Data     = ReceivedDir  + '\\SPF Coefficients.xlsx'

2018-09-07 09:49:54


In [3]:
# Reading Input Data
print(strftime("%Y-%m-%d %H:%M:%S"))
XML_DF     = hsmpy3.il.HSIP_DF_From_XML(XML_Dir)

HSIP_DF    = hsmpy3.il.CreateHSIPDataFrame(HSIP_Path,Years)
HSIP_DF = HSIP_DF[[(i in XML_DF.HSIPID) for i in HSIP_DF.index]]

WPPS_Ob_DF = hsmpy3.il.HSIP_DF_WPPS_Ob(WPPS_Ob_Path,HSIP_DF,XML_DF)
WPPS_Pr_DF = hsmpy3.il.HSIP_DF_WPPS_Pr(WPPS_Pr_Path,HSIP_DF,XML_DF)

HSIP_Joined = hsmpy3.il.Joined_HSIP_DF(XML_DF,HSIP_DF,WPPS_Ob_DF,WPPS_Pr_DF)

Cont_DF = hsmpy3.il.Joined_Contract_DF(HSIP_Joined,WPPS_Ob_DF,WPPS_Pr_DF,ReceivedDir,Years)

Loc_DF = hsmpy3.il.Joined_Loc_DF(WPPS_Pr_DF)

SPF_DF = pd.read_excel(SPF_Data,sheetname='Summary')
SPF_DF['PGNumber'] = [int(s.split('PeerGroup ')[1][:2]) for s in SPF_DF.PG]

PN = list(set(Loc_DF['ContNum']))
PN.sort()

2018-09-07 09:49:54


In [ ]:
# Create Results and Crash Type Figures
print(strftime("%Y-%m-%d %H:%M:%S"))
Conv_Dict = {
     'Animal':'Animal',
     'OtherNonCollision':'other non collision',
     'HeadOn':'head on',
     'FixedObj':'fixed object',
     'SideSwipeSameDir':'sideswipe same direction',
     'SideSwipeOppDir':'sideswipe opposite direction',
     'Pedestrian':'Pedestrian',
     'RearEnd':'Rear End',
     'OtherObj':'other object',
     'Overturned':'overturned',
     'Pedalcyclist':'pedestrian',
     'ParkedMotorVeh':'parked motor vehicle',
     'Turning':'turning',
     'Angle':'Angle'}
import warnings
warnings.filterwarnings('ignore')

XLSList = [os.path.join(ContractsDir, f) for f in os.listdir(ContractsDir) if os.path.isfile(os.path.join(ContractsDir, f)) and f.split('.')[1]=='xlsx' and 'HSIP_' in f.split('.')[0]]
print(len(XLSList))
for xls in XLSList:
    p = (os.path.basename(xls).split('_')[1].split('.')[0])
    df3 = pd.read_excel(xls,'RawCrash')
    df9 = pd.read_excel(xls,'BCDetails')
    df10 = pd.read_excel(xls,'BCSummary')
    tg = []
    hsipids = Cont_DF.loc[p]['HSIPIDs'].split(';')
    for hsipid in hsipids:
        if str(hsipid) in list(HSIP_Joined.index):
            l = HSIP_Joined.loc[str(hsipid)]['TargetCrashType'].split(';')
            for i in l:
                if i in Conv_Dict.keys():
                    tg.append(Conv_Dict[i])
    hsmpy3.il.PlotCrashType(df3,ContractsDir+'\\HSIP_' + str(p) + '_CrashType.png',p,tg)
    hsmpy3.il.PlotHSIP(df9,df10,p,ContractsDir+'\\HSIP_' + str(p) + '_Res.png')
    print(p)
print(strftime("%Y-%m-%d %H:%M:%S"))

In [5]:
# Create Maps and APRX
print(strftime("%Y-%m-%d %H:%M:%S"))
GDBList = [os.path.join(ContractsDir, f) for f in os.listdir(ContractsDir) if os.path.isdir(os.path.join(ContractsDir, f)) and f.split('.')[1]=='gdb' and 'HSIP_' in f.split('.')[0]]
print(len(GDBList))
for GDB in GDBList:
    p = os.path.basename(GDB).split('_')[1].split('.')[0]
    hsmpy3.il.CreateMapandProject(GDB,Years,APRX,Cont_DF,SegSymbology,IntSymbology,IntSymbology2,BeforeSymbology,AfterSymbology,ContractsDir)
    print(p)    

2018-09-07 16:09:13
696
13570
13677
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_13684_GIS.gdb\Seg_13684
13684
13872
13873
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_13937_GIS.gdb\Seg_13937
13937
13938
13940
14001
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_43475_GIS.gdb\Seg_43475
43475
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_43521_GIS.gdb\Seg_43521
43521
60A25
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_60A95_GIS.gdb\Seg_60A95
60A95
60C09
60C14
60C15
60D19
60D20
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_60D32_GIS.gdb\Seg_60D32
60D32
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Trackin

60W54
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_60W55_GIS.gdb\Seg_60W55
60W55
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_60W80_GIS.gdb\Seg_60W80
60W80
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_60W90_GIS.gdb\Seg_60W90
60W90
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_60W91_GIS.gdb\Seg_60W91
60W91
60W98
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_60W99_GIS.gdb\Seg_60W99
60W99
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_60X02_GIS.gdb\Seg_60X02
60X02
\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles\HSIP_60X08_GIS.gdb\Seg_60X08
60X08
\\CHCFPP01\Proj\ILDOT\